In [10]:
import pandas
from sklearn import model_selection
from sklearn.linear_model import LogisticRegression
import pickle
import codecs
import json
import gensim
import numpy as np
import math
import random
import numpy
import operator
import scipy
import Queue
from heapq import nlargest

In [23]:
def r_precision(G, R):
    limit_R = R[:len(G)]
    if len(G) != 0:
        return len(list(set(G).intersection(set(limit_R)))) * 1.0 / len(G)
    else:
        return 0

def ndcg(G, R):
    r = [1 if i in set(G).intersection(set(R)) else 0 for i in R]
    r = np.asfarray(r)
    dcg = r[0] + np.sum(r[1:] / np.log2(np.arange(2, r.size + 1)))
    #k = len(set(G).intersection(set(R)))
    k = len(G)
    if k > 0:
        idcg = 1 + np.sum(np.ones(k - 1) / np.log2(np.arange(2, k + 1)))
        return dcg * 1.0 / idcg
    else:
        return 0

def clicks(G, R):
    r = [1 if i in set(G).intersection(set(R)) else 0 for i in R]
    if sum(r) == 0:
        return 51
    else:
        return (r.index(1) - 1) * 1.0 / 10 

# Calculation Tools
# def find_nearest_k(distance, start, end, k):
#     pivot_index = random.randint(start, end)
#     original_start = start
#     original_end = end
#     while start <= end:
#         while start <= end and distance[start][1] <= distance[pivot_index][1]:
#             start += 1
#         while start <= end and distance[end][1] > distance[pivot_index][1]:
#             end -= 1
#         if start <= end:
#             distance[start], distance[end] = distance[end], distance[start]
#         else:
#             break
#     if start - original_start == k:
#         return distance[:start]
#     elif start - original_start > k:
#         return find_nearest_k(distance, original_start, start - 1, k)
#     else:
#         return find_nearest_k(distance, start, original_end, k - (start-original_start))

# def calculate_distance(pid, points):
#     global PL_Word2Vec_TEST
#     global TEST
#     global track_vec_touple
    
#     result = []
#     for track in track_vec_touple[:80]:
#         #if point[0] not in PL_TRACKS_TRAIN[pid]:
#         #print track
#         result.append((track[0], scipy.spatial.distance.cosine(np.array(PL_Word2Vec_TEST[pid]),np.array(track[1]))))
#     return result

# def find_nearest_k0(distances, k):
#     # Write your code here
#     switch_distances = dict((k, v) for v, k in distances.items())
#     res = []
#     if not distances or k == 0:
#         return res
    
#     MaxHeap = Queue.PriorityQueue()
    
#     # claculate distance
    
#     # end calculate distance

    
#     for key in distances:
#         MaxHeap.put(-distances[key])

#     for i in range(k):
#         res.append(switch_distances[-MaxHeap.get()])

#     return res

# def find_nearest_k1(pid, k):
#     global PL_Word2Vec_TEST
#     global TEST
    
#     res = []
#     if k == 0 or k > len(model.wv.vocab):
#         return res
    
#     MaxHeap = Queue.PriorityQueue()
     
#     # claculate distance
#     sim = {}
#     for track in model.wv.vocab.keys():
#         sim[track] = 1 - scipy.spatial.distance.cosine(np.array(PL_Word2Vec_TEST[pid]),np.array(model.wv[track]))
#         MaxHeap.put(-sim[track])

#     switch_sim = dict((k, v) for v, k in sim.items())    
#     for i in range(k):
#         res.append(switch_sim[-MaxHeap.get()])

#     return res

def find_nearest_k(pid, k):
    global PL_Word2Vec_TEST
    global TEST
    
    res = []
    sim = {}
    for track in model.wv.vocab.keys():
        sim[track] = 1 - scipy.spatial.distance.cosine(np.array(PL_Word2Vec_TEST[pid]),np.array(model.wv[track]))

    switch_sim = dict((k, v) for v, k in sim.items()) 
    sorted_sim = sorted(switch_sim.keys(), reverse = True)
    for i in range(k):
        res.append(switch_sim[sorted_sim[i]])

    return res

In [3]:
# Load Model
model = gensim.models.Word2Vec.load('../../Recsys2018/Model_0_JL/song2vec/song2vec_4_0')

In [24]:
TEST_GROUND_TRUTH_RAW = json.load(open('../DATA_PROCESSING/PL_TRACKS_5_TEST.json'))
TEST_GROUND_TRUTH = {}
for pl in TEST:
    TEST_GROUND_TRUTH[pl] = TEST_GROUND_TRUTH_RAW[pl]

In [4]:
# Calculate the average vector
track_sum_vec = np.zeros(100, dtype = float)
for track in model.wv.vocab:
    track_sum_vec += model.wv[track]

track_average_vec = track_sum_vec / len(model.wv.vocab)

In [28]:
type(track_average_vec)

numpy.ndarray

In [29]:
all_test_tracks = model.wv.vocab.keys()

In [5]:
task = 2
TEST = json.load(open('../DATA_PROCESSING/PL_TRACKS_5_TEST_T' + str(task) + '.json'))


In [6]:
PL_Word2Vec_TEST = {}
for pl in TEST:
    current = np.zeros(100)
    length = 0
    for track in TEST[pl]:
        if track in model.wv:
            current += model.wv[track]
        else:
            current += track_average_vec
        length += 1
    PL_Word2Vec_TEST[pl] = list(current / length)

In [26]:
R = {}
G = {}
for pl in TEST_GROUND_TRUTH:
    G[pl] = list(set(TEST_GROUND_TRUTH[pl]).difference(set(TEST[pl])))
    
for pl in TEST.keys()[:2]:
    R[pl] = find_nearest_k(pl, 500)
    
print 'Evaluating...'
print 'R_Precision:'

r_pre_result = []
for pl in TEST.keys()[:2]:
    r_pre_result.append(r_precision(G[pl], R[pl]))
r_pre_result = np.array(r_pre_result)

print '\tmean:', r_pre_result.mean(), 'std:', r_pre_result.std()

# with open('Model_0_Performance.txt', 'a') as f:
#     f.write('TASK_' + str(task) +' R_Precision_Mean '+str(r_pre_result.mean())+' R_Precision_Std '+ str(r_pre_result.std()) + '\n')


print 'NDCG:'
ndcg_result = []
for pl in TEST.keys()[:2]:
    ndcg_result.append(ndcg(G[pl], R[pl]))
ndcg_result = np.array(ndcg_result)
print '\tmean:', ndcg_result.mean(), 'std:', ndcg_result.std()

# with open('Model_0_Performance.txt', 'a') as f:
#     f.write('TASK_' + str(task) +' NDCG_Mean '+str(ndcg_result.mean())+' NDCG_Std '+ str(ndcg_result.std()) + '\n')


print 'Clicks:'
clicks_result = []
for pl in TEST.keys()[:2]:
    clicks_result.append(clicks(G[pl], R[pl]))
clicks_result = np.array(clicks_result)
print '\tmean:', clicks_result.mean(), 'std:', clicks_result.std()

# with open('Model_0_Performance.txt', 'a') as f:
#     f.write('TASK_' + str(task) +' Clicks_Mean '+str(clicks_result.mean())+' Clicks_Std '+ str(clicks_result.std()) + '\n')

print '========================================\n'

Evaluating...
R_Precision:
	mean: 0.3055555555555556 std: 0.19444444444444445
NDCG:
	mean: 0.4711674338523457 std: 0.280851563389977
Clicks:
	mean: 0.09999999999999999 std: 0.2

